In [1]:
import numpy as np
import pandas as pd

import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']
ind_rets = erk.get_ind_returns(weighting="vw", n_inds=49)["2013":"2018"][inds]
ind_mcap = erk.get_ind_market_caps(49, weights=True)["2013":"2018"][inds]

Weights

In [33]:
weights = erk.weight_cw(ind_rets, ind_mcap)
weights.sort_values(ascending=False)

Rtail    0.537344
Fin      0.184533
Food     0.139456
Whlsl    0.096043
Hlth     0.042624
Name: 2013-02, dtype: float64

Volatility

In [44]:
(np.diag(ind_rets.cov())*12)**0.5

array([0.16280055, 0.16775279, 0.13564368, 0.14123782, 0.12540067])

In [58]:
vols = pd.DataFrame(ind_rets.std()*np.sqrt(12))
vols

,0
Hlth,0.162801
Fin,0.167753
Whlsl,0.135644
Rtail,0.141238
Food,0.125401


In [34]:
rho = ind_rets.corr()
rho

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,1.000000,0.524886,0.650900,0.574227,0.371698
Fin,0.524886,1.000000,0.777653,0.595332,0.343016
Whlsl,0.650900,0.777653,1.000000,0.743754,0.541244
Rtail,0.574227,0.595332,0.743754,1.000000,0.575063
Food,0.371698,0.343016,0.541244,0.575063,1.000000


In [71]:
cov = ind_rets.cov()*12
cov

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.026504,0.014335,0.014374,0.013204,0.007588
Fin,0.014335,0.028141,0.017695,0.014105,0.007216
Whlsl,0.014374,0.017695,0.018399,0.014249,0.009206
Rtail,0.013204,0.014105,0.014249,0.019948,0.010185
Food,0.007588,0.007216,0.009206,0.010185,0.015725


In [68]:
sigma_prior = vols.dot(vols.T) * rho
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.026504,0.014335,0.014374,0.013204,0.007588
Fin,0.014335,0.028141,0.017695,0.014105,0.007216
Whlsl,0.014374,0.017695,0.018399,0.014249,0.009206
Rtail,0.013204,0.014105,0.014249,0.019948,0.010185
Food,0.007588,0.007216,0.009206,0.010185,0.015725


In [75]:
vols2 = ind_rets.std()*np.sqrt(12)
sigma_prior = vols2.dot(vols2.T) * rho

In [142]:
pi = 2.5 * sigma_prior.dot(weights).squeeze()
pi.sort_values()

Hlth     0.152854
Food     0.157528
Fin      0.176483
Whlsl    0.201786
Rtail    0.223770
dtype: float64

In [119]:
q = pd.Series([0.03])
# The Pick Matrix
p = pd.DataFrame([0.]*len(inds), index=inds).T
p[["Rtail", "Whlsl"]] = pd.DataFrame(-weights[["Rtail", "Whlsl"]] / weights[["Rtail", "Whlsl"]].sum()).T.set_index(p.index)
p['Hlth'] = 1.
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151635,-0.848365,0.0


In [127]:
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)


from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)


In [128]:
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=weights, sigma_prior=sigma_prior, p=p, q=q)
# Black Litterman Implied Mu
bl_mu

Hlth     0.179360
Fin      0.170206
Whlsl    0.193356
Rtail    0.199007
Food     0.144841
dtype: float64

In [129]:
# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w

In [134]:
np.round(w_msr(bl_sigma, bl_mu)*100, 2).sort_values(ascending=False)

Rtail    33.83
Hlth     27.72
Fin      18.45
Food     13.95
Whlsl     6.05
dtype: float64

In [138]:
q = pd.Series([0.05])
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151635,-0.848365,0.0


In [139]:
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = bl(w_prior=weights, sigma_prior=sigma_prior, p=p, q=q)
# Black Litterman Implied Mu
bl_mu.sort_values(ascending=False)

Rtail    0.193932
Whlsl    0.191629
Hlth     0.184792
Fin      0.168919
Food     0.142241
dtype: float64

In [137]:
np.round(w_msr(bl_sigma, bl_mu)*100, 2).sort_values(ascending=False)

Hlth     32.59
Rtail    29.70
Fin      18.45
Food     13.95
Whlsl     5.31
dtype: float64